In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
#Cargamos la base de datos 'dataset_unificado'
df_merged = pd.read_csv("C:/Users/guard/OneDrive/Desktop/Henry Data Science/Proyecto MLOps/Datasets Procesados/dataset_unificado.csv", sep=",")

df_merged.head(2)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western,genres_1,cast_names,directing_names
0,30000000.0,862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,0,0,0,0,0,0,0,"Animation, Comedy, FamilyAnimation, Comedy, Fa...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","John Lasseter, Roman Figun, Desirée Mourad"
1,65000000.0,8844,en,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,...,0,0,0,0,0,0,0,"Adventure, Fantasy, FamilyAdventure, Fantasy, ...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston


In [3]:
# Preprocesamiento de Datos - Utilizando 'title' para la recomendación
df_merged['title'] = df_merged['title'].fillna('')
# Vectorización de Textos - Usar TfidfVectorizer para transformar los títulos en vectores
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_merged['title'])
# Cálculo de Similaridad - Utilizar la similaridad del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [4]:
# Crear una serie con los títulos de las películas como índice
indices = pd.Series(df_merged.index, index=df_merged['title']).drop_duplicates()

In [12]:
def get_recommendations(title, cosine_sim, df_merged):
    # Obtener el índice de la película que coincide con el título
    idx = df_merged[df_merged['title'] == title].index[0]

    # Obtener las puntuaciones de similaridad para esa película
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas basadas en las puntuaciones de similaridad
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las puntuaciones de las 5 películas más similares
    sim_scores = sim_scores[1:6]

    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]

    # Devolver los títulos de las películas recomendadas
    return df_merged['title'].iloc[movie_indices]

# Ejemplo de uso: Obtener recomendaciones para una película específica
print(get_recommendations('Batman', cosine_sim, df_merged))

8611                       Batman
43735               Batman & Bill
44929    Batman Beyond: The Movie
18057            Batman: Year One
23199               Son of Batman
Name: title, dtype: object
